# Analyze parsed reports

This notebook analyzes the Texas Commission on Environmental Quality's Air Emission Event Reporting Database data parsed in the previous step. It focuses on reports of events since Aug. 23 in the Texas counties subject to the state's Harvey disaster declaration.

In [1]:
import pandas as pd

In [2]:
DISASTER_DECLARATION_COUNTIES = open("../inputs/disaster-declaration-counties.txt")\
    .read().strip()\
    .split("\n")
len(DISASTER_DECLARATION_COUNTIES)

54

## Load the data

In [3]:
reports = pd.read_csv(
    "../outputs/report-metadata-raw.csv",
)
reports["Event began"] = pd.to_datetime(reports["Event began"])
reports["Event ended"] = pd.to_datetime(reports["Event ended"], errors="coerce")

In [4]:
reports.head()

,Action taken,Cause,"City, County",Emissions estimation method,Event began,Event ended,Physical location,Regulated entity RN number,Regulated entity name,This is based on the,Type(s) of air emissions event,report_id
0,NaN,Unauthorized discharge at 150 Persimmon Manhol...,"BAYTOWN, HARRIS",NaN,2017-08-10 20:43:00,2017-08-10 23:45:00,NaN,RN101611457,EAST DISTRICT,FINAL REPORT,WASTEWATER BYPASS,265500
1,NaN,Chlorinated Excursion; Cleared Private Line; C...,"HOUSTON, HARRIS",NaN,2017-08-04 00:00:00,2017-08-04 00:00:00,NaN,RN101607596,BELTWAY WWTP,FINAL REPORT,WASTEWATER BYPASS,265502
2,NaN,Scheduled for Further Repairs;,"HOUSTON, HARRIS",NaN,2017-08-04 00:00:00,2017-08-04 00:00:00,NaN,RN101612158,FWSD 23 WWTP,FINAL REPORT,WASTEWATER BYPASS,265503
3,NaN,Unauthorized Discharge at 1016 Applewood manho...,"FRIENDSWOOD, HARRIS",NaN,2017-08-14 19:00:00,2017-08-14 22:00:00,NaN,RN102183340,BLACKHAWK REGIONAL WTP,FINAL REPORT,WASTEWATER BYPASS,265504
4,NaN,Chlorinated Excursion; Cleared Private Line; C...,"HOUSTON, HARRIS",NaN,2017-08-04 00:00:00,2017-08-04 00:00:00,"9400 White Chapel Ln, Houston, TX",RN101614113,KEEGANS BAYOU WWTP,FINAL REPORT,WASTEWATER BYPASS,265505


## Separate city and county

In [5]:
reports["City"] = reports["City, County"].apply(lambda x: x.split(", ")[0])
reports["City"].value_counts().head(10)

                  180
HOUSTON           144
SAN ANTONIO        47
GOLDSMITH          35
CRANE              33
CORPUS CHRISTI     23
AUSTIN             19
PASADENA           19
PORT ARTHUR        18
BAYTOWN            15
Name: City, dtype: int64

In [6]:
reports["County"] = reports["City, County"].apply(lambda x: x.split(", ")[1])
reports["County"].value_counts().head(10)

HARRIS       244
BEXAR         49
CRANE         39
ECTOR         38
JEFFERSON     36
GALVESTON     33
TARRANT       29
NUECES        28
BRAZORIA      27
DALLAS        25
Name: County, dtype: int64

### Here are the counties we're not analyzing, as a data-check

In [7]:
print("\n".join(sorted(reports[~reports["County"].str.upper().isin(DISASTER_DECLARATION_COUNTIES)]["County"].unique())))

ANDERSON
ANDREWS
ANGELINA
BELL
BOSQUE
BOWIE
BURNET
CAMP
CASS
CASTRO
CHEROKEE
COCHRAN
COLLIN
COMANCHE
CRANE
CULBERSON
DALLAS
DAWSON
DENTON
DIMMIT
DUVAL
EASTLAND
ECTOR
EL PASO
ELLIS
ERATH
FALLS
FANNIN
FREESTONE
FRIO
GAINES
GLASSCOCK
GRAYSON
HARRISON
HAYS
HENDERSON
HIDALGO
HILL
HOCKLEY
HOOD
HOUSTON
HOWARD
HUNT
HUTCHINSON
JOHNSON
JONES
KAUFMAN
KENT
LA SALLE
LIMESTONE
LUBBOCK
MARTIN
MASON
MCLENNAN
MCMULLEN
MIDLAND
MILAM
MITCHELL
MONTAGUE
NACOGDOCHES
NOLAN
ORANGE
PALO PINTO
PANOLA
PARKER
PECOS
POTTER
RANDALL
REAGAN
REEVES
ROCKWALL
RUNNELS
RUSK
SABINE
SAN AUGUSTINE
SCHLEICHER
SHELBY
SHERMAN
SMITH
STEPHENS
TARRANT
TAYLOR
TITUS
TOM GREEN
TRAVIS
TRINITY
UPTON
VAN ZANDT
WARD
WEBB
WHEELER
WILBARGER
WILLIAMSON
WINKLER
WISE
YOAKUM


## Count by type

In [8]:
REPORT_IDS_TO_IGNORE = [ int(line.split("#")[0].strip())
    for line in open("../inputs/reports-to-ignore.txt") ]
REPORT_IDS_TO_IGNORE

[266113,
 266073,
 266136,
 266156,
 266246,
 267665,
 266648,
 267063,
 266274,
 266338,
 266966,
 267240,
 267389,
 267394,
 267398,
 267624,
 266675,
 267677,
 267253,
 266301,
 266266]

In [9]:
reports_of_interest = reports[
    reports["County"].str.upper().isin(DISASTER_DECLARATION_COUNTIES) &
    ~reports["report_id"].isin(REPORT_IDS_TO_IGNORE) &
    (reports["Event began"] >= "2017-08-23") &
    (reports["Type(s) of air emissions event"].isin([ "EMISSIONS EVENT", "AIR SHUTDOWN", "AIR STARTUP" ]))
]

## Summarize report data

In [10]:
reports_of_interest["Regulated entity RN number"].nunique()

53

In [11]:
reports_of_interest.groupby([ "County", "Type(s) of air emissions event" ])\
    .size()\
    .unstack()\
    .fillna(0)\
    .astype(int)\
    .assign(total=lambda x: x.sum(axis=1))\
    .sort_values(["total"], ascending=False)

Type(s) of air emissions event,AIR SHUTDOWN,AIR STARTUP,EMISSIONS EVENT,total
County,,,,
HARRIS,3,2,27,32
JEFFERSON,4,5,12,21
NUECES,2,3,12,17
BRAZORIA,1,2,7,10
CHAMBERS,0,3,4,7
CALHOUN,1,3,1,5
VICTORIA,1,4,0,5
GALVESTON,1,1,2,4
GRIMES,0,0,2,2


## Analyze contaminants emitted

In [12]:
emissions = pd.read_csv("../outputs/report-emissions-raw.csv")\
    .pipe(lambda x: x[x["report_id"].isin(reports_of_interest["report_id"])])
emissions.head()

,report_id,contaminant,authorization,limit,amount_released
735,266106,Opacity,NSR Permit 813,20.0 % op,6.3 % op (est.)
736,266106,Opacity,NSR Permit 812,20.0 % op,0.55 % op (est.)
737,266106,Opacity,NSR Permit 812,20.0 % op,7.7 % op (est.)
743,266110,Acetylene,19168/PSDTX1226,761.65 LBS/HR,90.77 lbs (est.)
744,266110,Benzene,19168/PSDTX1226,761.65 LBS/HR,147.1 lbs (est.)


In [13]:
emissions["quantity"] = emissions["amount_released"].apply(lambda x: x.split(" ", 1)[0])\
    .replace("Unknown", pd.np.nan)\
    .astype(float)
emissions["quantity"].head()

735      6.30
736      0.55
737      7.70
743     90.77
744    147.10
Name: quantity, dtype: float64

In [14]:
emissions["units"] = emissions["amount_released"].apply(lambda x: x.split(" ", 1)[1] if " " in x else None)\
    .replace("Unknown", pd.np.nan)
emissions["units"].head()

735    % op (est.)
736    % op (est.)
737    % op (est.)
743     lbs (est.)
744     lbs (est.)
Name: units, dtype: object

In [15]:
emissions["units"].value_counts()

lbs (est.)     1453
% op (est.)      21
lbs               2
LBS/HR            1
Name: units, dtype: int64

In [16]:
EMISSIONS_SUMMARY_COLS = [ 
    "report_id",
    "Event began",
    "Event ended",
    "Regulated entity RN number",
    "Regulated entity name",
    "Type(s) of air emissions event",
    "County",
    "contaminant",
    "authorization",
    "limit",
    "quantity",
    "units",
]

In [17]:
emissions_lbs = emissions[
    emissions["units"].str.contains(r"\blbs")
].pipe(pd.merge, reports, on="report_id")\
    .sort_values(["quantity", "report_id", "contaminant"], ascending=False)\
    [EMISSIONS_SUMMARY_COLS]
    
emissions_lbs.to_csv("../outputs/largest-emissions-in-lbs.csv", index=False)

In [18]:
emissions_lbs[
    emissions_lbs["quantity"] > 0
]["Regulated entity RN number"].nunique()

48

In [19]:
emissions_lbs[
    emissions_lbs["quantity"] > 0
]["report_id"].nunique()

95

In [20]:
emissions_lbs.head()

,report_id,Event began,Event ended,Regulated entity RN number,Regulated entity name,Type(s) of air emissions event,County,contaminant,authorization,limit,quantity,units
162,266261,2017-08-27 00:00:00,2017-09-06 00:00:00,RN103919817,CHEVRON PHILLIPS CHEMICAL CEDAR BAYOU PLANT,AIR SHUTDOWN,HARRIS,Carbon Monoxide,1504A,1892.04 LBS/HR,244040.0,lbs (est.)
786,266378,2017-08-29 11:50:00,2017-09-01 11:50:00,RN100217389,FLINT HILLS RESOURCES PORT ARTHUR FACILITY,AIR SHUTDOWN,JEFFERSON,Carbon Monoxide,No specific Authorization,0.0,240000.0,lbs (est.)
1381,267078,2017-09-08 08:00:00,2017-09-22 08:00:00,RN100217389,FLINT HILLS RESOURCES PORT ARTHUR FACILITY,AIR STARTUP,JEFFERSON,Ethylene (gaseous),No specific authorization,0.0,150000.0,lbs (est.)
1379,267078,2017-09-08 08:00:00,2017-09-22 08:00:00,RN100217389,FLINT HILLS RESOURCES PORT ARTHUR FACILITY,AIR STARTUP,JEFFERSON,Carbon Monoxide,No specific authorization,0.0,150000.0,lbs (est.)
1120,266566,2017-09-01 05:00:00,2017-10-01 05:00:00,RN100221662,EQUISTAR CORPUS CHRISTI PLANT,AIR STARTUP,NUECES,Carbon Monoxide,Permit 83864,721.67 LBS/HR,121000.0,lbs (est.)


In [21]:
emissions_lbs["quantity"].sum()

5419902.4145000009

## Analyze emissions by facility

In [22]:
emissions_lbs.groupby([ "Type(s) of air emissions event" ])["quantity"].sum() / 1e6

Type(s) of air emissions event
AIR SHUTDOWN       1.923592
AIR STARTUP        1.890128
EMISSIONS EVENT    1.606182
Name: quantity, dtype: float64

In [23]:
lbs_grp = emissions_lbs.groupby([ "Regulated entity RN number" ])

lbs_by_entity = pd.DataFrame({
    "Regulated entity name": lbs_grp["Regulated entity name"].first(),
    "County": lbs_grp["County"].first(),
    "quantity_1000s": (lbs_grp["quantity"].sum() / 1000).round(2),
    "contaminants": lbs_grp["contaminant"].apply(lambda x: " • ".join(sorted(x.str.lower().unique()))),
    "report_ids": lbs_grp["report_id"].apply(lambda x: " • ".join(sorted(x.astype(str).str.lower().unique()))),
    "units": "lbs"
}).sort_values(["quantity_1000s", "Regulated entity name"], ascending=False)

lbs_by_entity.to_csv("../outputs/facilities-with-most-emissions-lbs.csv")
lbs_by_entity

,County,Regulated entity name,contaminants,quantity_1000s,report_ids,units
Regulated entity RN number,,,,,,
RN100217389,JEFFERSON,FLINT HILLS RESOURCES PORT ARTHUR FACILITY,"1,3-butadiene • acetylene • benzene • butanes ...",1010.70,266378 • 267078 • 267266,lbs
RN103919817,HARRIS,CHEVRON PHILLIPS CHEMICAL CEDAR BAYOU PLANT,"1,3-butadiene • acetylene • benzene • butane •...",745.47,266261 • 266262,lbs
RN100210608,GALVESTON,MARATHON PETROLEUM TEXAS CITY REFINERY,"ammonia • benzene • butadiene • butane, i • bu...",396.55,266376 • 266463 • 266570,lbs
RN102579307,HARRIS,EXXON MOBIL BAYTOWN REFINERY,"1,3-butadiene • ammonia • benzene • butane • b...",383.85,266277 • 266294,lbs
RN100825249,BRAZORIA,CHEVRON PHILLIPS CHEMICAL SWEENY OLD OCEAN FAC...,"1,3-butadiene • acetylene • benzene • butane, ...",365.01,266372 • 267392,lbs
RN102584026,JEFFERSON,VALERO PORT ARTHUR REFINERY,carbon monoxide • hydrogen sulfide • oxides of...,354.29,266764 • 266817 • 266834 • 267400,lbs
RN100238708,BRAZORIA,CHOCOLATE BAYOU PLANT,"1,3-butadiene • butene • carbon monoxide • eth...",318.55,266271 • 266897 • 267010,lbs
RN100221662,NUECES,EQUISTAR CORPUS CHRISTI PLANT,"1,3-butadiene • acetylene • benzene • butane •...",313.15,266177 • 266566,lbs
RN100224815,HARRIS,PASADENA TERMINAL,"1 c,2t,3-trimethylcyclopentane • 1,2,4-trimeth...",166.06,266269 • 266556,lbs


# Analyze 2015 and 2016 emissions from these same facilities

In [24]:
historical_report_metadata = pd.read_csv("../outputs/report-metadata-raw-historical.csv")
historical_report_emissions = pd.read_csv("../outputs/report-emissions-raw-historical.csv")

Emissions in millions of pounds:

In [25]:
pd.merge(
    historical_report_metadata,
    historical_report_emissions,
    how="left",
    on="report_id"
)\
    .pipe(lambda x: x[x["amount_released"].str.contains(r"\blbs", na=False)])\
    .assign(year=lambda x: pd.to_datetime(x["Event began"]).dt.year)\
    .assign(amt=lambda x: x["amount_released"].apply(lambda x: float(x.split(" ")[0])))\
    .groupby(["year", "This is based on the"])["amt"].sum().unstack() / 1e6

This is based on the,FINAL REPORT,INITIAL REPORT
year,,
2014,0.287886,NaN
2015,8.158067,0.064122
2016,5.493888,0.046722


---

---

---